In [1]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py

In [2]:
%matplotlib widget

### Discharge data

In [3]:
step_cal_dir = r'/data/new_trap/20210823/Bead2/Discharge/FineDischarge/'
step_cal_files = load_dir(step_cal_dir, start_file=130)

100%|██████████| 30/30 [00:00<00:00, 40.06it/s]

160  files in folder
30  files loaded


In [4]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
fsamp = 5000
def func1_scale(i):
    bb = step_cal_files[i]
        
    freq2 = np.fft.rfftfreq(len(bb.electrode_data[0]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.electrode_data[0]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.electrode_data[0])*np.sqrt(2))

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0],m1_tmp.values[2]

In [5]:
amps3 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i)[0])

0 / 30
[0.08151204] 9.996282374187803
1 / 30
[0.08150584] 9.996283154554497
2 / 30
[0.08151288] 9.996289306700985
3 / 30
[0.08150226] 9.996290922600725
4 / 30
[0.08149532] 9.996283225508078
5 / 30
[0.0815016] 9.996280947525667
6 / 30
[0.0815034] 9.996280890678362
7 / 30
[0.08151182] 9.996282502789684
8 / 30
[0.08152733] 9.996280499973766
9 / 30
[0.08151452] 9.996285141151462
10 / 30
[0.08148874] 9.996282313841002
11 / 30
[0.08148649] 9.996282478421824
12 / 30
[0.08149802] 9.996287513392774
13 / 30
[0.08150148] 9.996283785231162
14 / 30
[0.08148665] 9.99628420650162
15 / 30
[0.08150219] 9.996287111134162
16 / 30
[0.08149921] 9.996282544948206
17 / 30
[0.08150441] 9.996285968052252
18 / 30
[0.08151877] 9.996289006229093
19 / 30
[0.08153172] 9.996289370541222
20 / 30
[0.08152797] 9.996283176744285
21 / 30
[0.08152914] 9.996285002860057
22 / 30
[0.08153586] 9.99628791383331
23 / 30
[0.08152052] 9.996288159636872
24 / 30
[0.08152625] 9.996286647416502
25 / 30
[0.08152876] 9.996280108323218


In [6]:
scale = np.abs(np.mean(amps3[70:80]))/8
print("Scale is ", scale)
bb = step_cal_files[-1]
voltage = np.std(bb.electrode_data[0])*np.sqrt(2)
print("voltage: ", voltage)

Scale is  nan
voltage:  9.996283860738322


/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/analysis_user/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
scale=2.1e-5
amps3 = np.array(amps3)
_,ax = plt.subplots(figsize=(9.4,4))

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

ax.set(xlabel='dataset', ylabel='Electron charges',)# ylim=(-22, 0.1), xlim=(30,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
phase_calib = 0
# phase_calib = -0.0442

def get_amp(i):
    
    fname = r'/data/new_trap/20210819/Bead1/Neutrality/200V_X_SPINYZ_58p5kHz_100V_20210820/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)


    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phase_calib, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}
    
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [9]:
i=0
get_amp(i)/scale

-0.05256970605916847

In [10]:
i = 0
fname = r'/data/new_trap/20210819/Bead1/Neutrality/200V_X_SPINYZ_58p5kHz_100V_20210820/Discharge_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
voltage0_200 = np.std(bb.electrode_data[0])*np.sqrt(2)*200
voltage1_200 = np.std(bb.electrode_data[1])*np.sqrt(2)*200
bias2 = np.mean(bb.electrode_data[0])*200
print("voltages, bias: ", voltage0_200, voltage1_200, bias2)

voltages, bias:  200.04036193179343 0.04137692997338144 0.42040024668424963


In [11]:
voltage, voltage0_200, scale

(9.996283860738322, 200.04036193179343, 2.1e-05)

In [12]:
amps_data_200V = np.array(Parallel(n_jobs=20)(delayed(get_amp)(i) for i in tqdm(range(1800))))

100%|██████████| 1800/1800 [00:56<00:00, 31.19it/s]


In [ ]:
data0 = amps_data_200V/scale*voltage/voltage0_200

dataPlus=data0[::4]
dataMinus=data0[2::4]

_, ax= plt.subplots()
ax.scatter(np.arange(len(dataPlus)), dataPlus, label='X-axis, plus, 200V, YZ-spin, with-height-fb')
ax.scatter(np.arange(len(dataMinus)), dataMinus, label='X-axis, minus, 200V, YZ-spin, with-height-fb')
ax.plot(np.arange(0,1000), np.arange(0,1000)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean: ', np.mean(dataPlus), 'std: ', np.std(dataPlus)/np.sqrt(len(dataPlus)))
print('mean: ', np.mean(dataMinus), 'std: ', np.std(dataMinus)/np.sqrt(len(dataMinus)))

In [ ]:
-0.007713032166712999/0.03923405818351312 

In [ ]:
-0.0016038530141878442/0.007201926298527419